### Goal:
- get the basic understandign of how sagemaker_pyspark works

In [1]:
pip install sagemaker_pyspark

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.


In [8]:
! pip install -qU sagemaker

### Load SageMaker Jars programatically

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession

import sagemaker_pyspark

conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars())))
sc=SparkContext(conf=conf)
spark=SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 07:21:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

arn:aws:iam::432547830124:role/service-role/AmazonSageMaker-ExecutionRole-20221230T201554


### Creating Spark Training Job

In [10]:
from sagemaker_pyspark import IAMRole
from sagemaker_pyspark.algorithms import KMeansSageMakerEstimator

iam_role = "arn:aws:iam::432547830124:role/service-role/AmazonSageMaker-ExecutionRole-20221230T201554"

region = "us-east-1"
training_data = spark.read.format("libsvm").option("numFeatures", "784")\
  .load("s3a://sagemaker-sample-data-{}/spark/mnist/train/".format(region))

test_data = spark.read.format("libsvm").option("numFeatures", "784")\
  .load("s3a://sagemaker-sample-data-{}/spark/mnist/train/".format(region))

kmeans_estimator = KMeansSageMakerEstimator(
    trainingInstanceType="ml.m4.xlarge",
    trainingInstanceCount=1,
    endpointInstanceType="ml.m4.xlarge",
    endpointInitialInstanceCount=1,
    sagemakerRole=IAMRole(iam_role))

kmeans_estimator.setK(10)
kmeans_estimator.setFeatureDim(784)

kmeans_model = kmeans_estimator.fit(training_data)

transformed_data = kmeans_model.transform(test_data)
transformed_data.show()

23/01/02 07:29:20 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
23/01/02 07:29:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
23/01/02 07:29:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


+-----+--------------------+-------------------+---------------+
|label|            features|distance_to_cluster|closest_cluster|
+-----+--------------------+-------------------+---------------+
|  5.0|(784,[152,153,154...| 1663.5850830078125|            2.0|
|  0.0|(784,[127,128,129...|  1216.953369140625|            5.0|
|  4.0|(784,[160,161,162...| 1845.0809326171875|            9.0|
|  1.0|(784,[158,159,160...|     1401.056640625|            3.0|
|  9.0|(784,[208,209,210...| 1467.3529052734375|            9.0|
|  2.0|(784,[155,156,157...| 1716.1195068359375|            6.0|
|  1.0|(784,[124,125,126...|     1403.333984375|            3.0|
|  3.0|(784,[151,152,153...|  1632.590576171875|            2.0|
|  1.0|(784,[152,153,154...| 1153.1788330078125|            3.0|
|  4.0|(784,[134,135,161...|      1453.21484375|            0.0|
|  3.0|(784,[123,124,125...|  1507.247314453125|            2.0|
|  5.0|(784,[216,217,218...|   1611.57763671875|            3.0|
|  3.0|(784,[143,144,145.

#### References  
https://sagemaker-pyspark.readthedocs.io/en/latest/